# Coin

A - монета неправильная

Bn - выпало n орлов
$$P(A|B_n) = \frac{P(A \cap B_n)}{P(B_n)} = \frac{P(B_n | A) * P(A)}{P(B_n | A) * P(A) + P(B_n | \neg A) * P(\neg A)} = \frac{0.9^{n} * 0.5}{0.9^{n} * 0.5 + 0.5^{n} * 0.5} \geqslant 0.95 $$

$$ 0.9^n \geqslant 0.95 * (0.9^n + 0.5^n) $$

$$ \frac{0.9^n}{20} \geqslant 0.95 * 0.5^n$$

$$ 1.8^n \geqslant 19 \Rightarrow n \geqslant 6 $$

# PAM

$$ PAM^1 = 
\begin{bmatrix} 
1-3a & a & a & a \\
a & 1-3a & a & a \\
a & a & 1-3a & a \\
a & a & a & 1-3a  \\
\end{bmatrix}
$$

Тогда, исходя из симметрии:
$$ PAM^k = 
\begin{bmatrix} 
1-3a_k & a_k & a_k & a_k \\
a_k & 1-3a_k & a_k & a_k \\
a_k & a_k & 1-3a_k & a_k \\
a_k & a_k & a_k & 1-3a_k  \\
\end{bmatrix}
$$

$$ PAM^{k + 1} = PAM^k * PAM ^ 1 \Rightarrow a_{k+1} = (1 - 3a) * a_k + a(1 - 3a_k) + aa_k + aa_k = a_k(1 - 4a) + a$$

$$ a_1 = a$$

$$ a_2 = a(1 - 4a) + a = 2a - 4a^2 = \frac{1}{4} - \frac{1}{4} + 2a - 4a^2 = \frac{1}{4}(1 - (1 - 8a +16a^2)) = \frac{1}{4}(1 - (1 - 4a)^2)$$

$$ a_3 = \frac{1}{4}(1 - (1 - 4a)^2) * (1 - 4a) + a = \frac{1}{4}(1 - 4a - (1 - 4a)^3 + 4a) = \frac{1}{4}(1 - (1 - 4a)^3) $$



Докажем с помощью мат индукции, что зависимость имеет вид 
$$ a_n = \frac{1}{4}(1 - (1 - 4a)^n) $$
База индукции - выведенные ранее $$ a_1, a_2, a_3$$
ШИ $$ a_{n + 1} = \frac{1}{4}(1 - (1 - 4a)^n) * (1 - 4a) + a = \frac{1}{4}(1 - 4a - (1 - 4a)^{n + 1} + 4a) = \frac{1}{4}(1 - (1 - 4a)^{n + 1}) $$



Значит
$$ a_n = \frac{1}{4}(1 - (1 - 4a)^n) $$

$$ 1 - 3a_n = 1 - \frac{3}{4}(1 - (1 - 4a)^n) = \frac{3(1 - 4a)^n + 1}{4} $$ 



$$ PAM^n = \frac{1}{4} * \begin{bmatrix} 
3(1 - 4a)^n + 1 & 1 - (1 - 4a)^n & 1 - (1 - 4a)^n & 1 - (1 - 4a)^n \\
1 - (1 - 4a)^n & 3(1 - 4a)^n + 1 & 1 - (1 - 4a)^n & 1 - (1 - 4a)^n \\
1 - (1 - 4a)^n & 1 - (1 - 4a)^n & 3(1 - 4a)^n + 1 & 1 - (1 - 4a)^n \\
1 - (1 - 4a)^n & 1 - (1 - 4a)^n & 1 - (1 - 4a)^n & 3(1 - 4a)^n + 1  \\
\end{bmatrix}
$$

# Viterbi + FB

In [1]:
import numpy as np
from collections import namedtuple

Probabilities = namedtuple('Probabilities', ('p', 'a11', 'a12', 'a21', 'a22', 'b1', 'b2'))

In [117]:
def viterbi(seq, t):
    n = len(seq)
    d = []
    l = t.p * t.b1[seq[0]]
    m = (1 - t.p) * t.b2[seq[0]]

    for i in range(1, n):
        sym = seq[i] 
        first_from_first_state = l * t.a11 * t.b1[sym]
        first_from_second_state = m * t.a21 * t.b1[sym]
        second_from_first_state = l * t.a12 * t.b2[sym]
        second_from_second_state = m * t.a22 * t.b2[sym]
        step1 = first_from_second_state > first_from_first_state
        l = first_from_second_state if step1 else first_from_first_state
        step2 = second_from_second_state > second_from_first_state 
        m = second_from_second_state if step2 else second_from_first_state
        
        last = (step1, step2)
        d.append(last)
    print(d)
    
    step = l < m
    trail = [int(step) + 1]
    for i in range(n - 1):
        step = d.pop()[step]
        trail.append(int(step) + 1)
    return list(reversed(trail))

In [118]:
def fb(seq, t):
    n = len(seq)
    a, b = np.zeros((2, n)), np.zeros((2, n))
    a[0, 0] = t.p * t.b1[seq[0]]
    a[1, 0] = t.p * t.b2[seq[0]]
    b[0, n - 1] = 1
    b[1, n - 1] = 1
    for i in range(1, n):
        sym = seq[i]
        a1, a2 = a[0, i - 1], a[1, i - 1] 
        a[0, i] = a1 * t.a11 * t.b1[sym] + a2 * t.a21 * t.b1[sym]
        a[1, i] = a1 * t.a12 * t.b2[sym] + a2 * t.a22 * t.b2[sym]
        
        sym = seq[n - i]
        b1, b2 = b[0, n - i], b[1, n - i] 
        b[0, n - i - 1] = b1 * t.a11 * t.b1[sym] + b2 * t.a12 * t.b2[sym]
        b[1, n - i - 1] = b1 * t.a21 * t.b1[sym] + b2 * t.a22 * t.b2[sym]
    p1 = [round(a[0, i] * b[0, i] / (a[0, n - 1] + a[1, n - 1]), 3) for i in range(n)]
    p2 = [round(a[1, i] * b[1, i] / (a[0, n - 1] + a[1, n - 1]), 3) for i in range(n)]
    return p1, p2

## Test1

In [119]:
sequence = 'OPOPOPOOPPPPPPPPPPOOOOOOOO'

# choose first initial hidden state
p = 0.5
# change hidden state
a11 = 0.8
a12 = 0.2
a22 = 0.8
a21 = 0.2

# emit in first hidden state
b1 = {'O': 0.5, 'P': 0.5}

# emit in second hidden state
b2 = {'O': 0.1, 'P': 0.9}

prob = Probabilities(p, a11, a12, a21, a22, b1, b2)

In [120]:
res = viterbi(sequence, prob)
trace = fb(sequence, prob)
print("Viterbi:")
print(old_viterbi(sequence, prob))
print(res)
print(len(res))
print('FB:')
for v in trace:
    print(v)

0.25 0.05
1
1 -> 1: 0.1
2 -> 1: 0.005000000000000001
1 -> 2: 0.045000000000000005
2 -> 2: 0.03600000000000001

0.1 0.045000000000000005
2
1 -> 1: 0.04000000000000001
2 -> 1: 0.0045000000000000005
1 -> 2: 0.0020000000000000005
2 -> 2: 0.0036000000000000008

0.04000000000000001 0.0036000000000000008
3
1 -> 1: 0.016000000000000004
2 -> 1: 0.0003600000000000001
1 -> 2: 0.0072000000000000015
2 -> 2: 0.0025920000000000006

0.016000000000000004 0.0072000000000000015
4
1 -> 1: 0.006400000000000002
2 -> 1: 0.0007200000000000002
1 -> 2: 0.00032000000000000013
2 -> 2: 0.0005760000000000001

0.006400000000000002 0.0005760000000000001
5
1 -> 1: 0.002560000000000001
2 -> 1: 5.760000000000002e-05
1 -> 2: 0.0011520000000000005
2 -> 2: 0.00041472000000000015

0.002560000000000001 0.0011520000000000005
6
1 -> 1: 0.0010240000000000004
2 -> 1: 0.00011520000000000005
1 -> 2: 5.1200000000000025e-05
2 -> 2: 9.216000000000004e-05

0.0010240000000000004 9.216000000000004e-05
7
1 -> 1: 0.0004096000000000002
2 -

## Test2

In [121]:
sequence = 'OPOPOPOOPPPPPPPPPPOOOOOOOO'

# choose first initial hidden state
p = 0.5
# change hidden state
a11 = 0.5
a12 = 0.5
a22 = 0.5
a21 = 0.5

# emit in first hidden state
b1 = {'O': 0.5, 'P': 0.5}

# emit in second hidden state
b2 = {'O': 0.51, 'P': 0.49}

prob2 = Probabilities(p, a11, a12, a21, a22, b1, b2)

In [122]:
res = viterbi(sequence, prob2)
trace = fb(sequence, prob2)
print("Viterbi:")

print(old_viterbi(sequence, prob2))
print(res)
print('FB:')
for v in trace:
    print(v)

0.25 0.255
1
1 -> 1: 0.0625
2 -> 1: 0.06375
1 -> 2: 0.06125
2 -> 2: 0.062475

0.06375 0.062475
2
1 -> 1: 0.0159375
2 -> 1: 0.01561875
1 -> 2: 0.01625625
2 -> 2: 0.015931125

0.0159375 0.01625625
3
1 -> 1: 0.003984375
2 -> 1: 0.0040640625
1 -> 2: 0.0039046875
2 -> 2: 0.00398278125

0.0040640625 0.00398278125
4
1 -> 1: 0.001016015625
2 -> 1: 0.0009956953125
1 -> 2: 0.0010363359375
2 -> 2: 0.00101560921875

0.001016015625 0.0010363359375
5
1 -> 1: 0.00025400390625
2 -> 1: 0.000259083984375
1 -> 2: 0.000248923828125
2 -> 2: 0.00025390230468749997

0.000259083984375 0.00025390230468749997
6
1 -> 1: 6.477099609375e-05
2 -> 1: 6.347557617187499e-05
1 -> 2: 6.6066416015625e-05
2 -> 2: 6.47450876953125e-05

6.477099609375e-05 6.6066416015625e-05
7
1 -> 1: 1.61927490234375e-05
2 -> 1: 1.651660400390625e-05
1 -> 2: 1.651660400390625e-05
2 -> 2: 1.6846936083984375e-05

1.651660400390625e-05 1.6846936083984375e-05
8
1 -> 1: 4.129151000976562e-06
2 -> 1: 4.211734020996094e-06
1 -> 2: 4.0465679809570

26
26
